# Machine Learning Service - AML ComputeでPytorch MNISTを実行する

## SDKのインストール
Azure Notebookで実行する際は、SDKのインストールから開始することができます。
自分のJupyter環境を利用する場合は、別途環境設定が必要です。
詳細は下記をご参照ください。
https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-configure-environment

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

C:\Users\yoo\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


SDK version: 0.1.74


## 診断ツールのインストール

In [18]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

## ワークスペースの作成
Azure Machine Learning Serviceでは、ワークスペースという管理単位でジョブやクラスターなどを管理します。また、最初にワークスペースを作成する際には、ワークスペースを含むリソースグループの中に、Azure Storage Account, Azure Container Registry, Azure Application Insightsが一緒に作成され、裏側で連携するようになっています。
下記のいずれかから、ワークスペースを作成・設定します。なお、下記のプロセスではAzure ADの認証が発生する場合があります。

In [2]:
# ワークスペースを新規に作成する
# ワークスペースの作成には引数として、名前、サブスクリプションID、リソースグループ名、ロケーション（リージョン）が必要です。
from azureml.core.workspace import Workspace

ws = Workspace.create(name='WS-EastUS',
                      subscription_id='<サブスクリプションID>',
                      resource_group='ML-EastUS',
                      create_resource_group=True,
                      location='eastus' # or other supported Azure region
                     )

WorkspaceException: You have access to more than one subscriptions. Please specify one from this list = [SubscriptionInfo(subscription_name='Microsoft Azure 社内従量課金プラン', subscription_id='d36d3e26-1efc-45c9-9af2-b4c3fc712ef9'), SubscriptionInfo(subscription_name='Cost Management Demo', subscription_id='1caaa5a3-2b66-438e-8ab4-bce37d518c5d'), SubscriptionInfo(subscription_name='Services Hub Demo Open', subscription_id='1b0a5c20-7373-41a2-8fec-7364cceb7bbf'), SubscriptionInfo(subscription_name='Contoso IT – SH360 – Prod', subscription_id='e93d3ee6-fac1-412f-92d6-bfb379e81af2'), SubscriptionInfo(subscription_name='Contoso IT - demo', subscription_id='e4272367-5645-4c4e-9c67-3b74b59a6982'), SubscriptionInfo(subscription_name='Contoso IT – Retail - DevTest', subscription_id='35ad74f4-0b37-44a7-ba94-91b6ec6026cd'), SubscriptionInfo(subscription_name='Contoso Infra3', subscription_id='66ee64da-2532-4d9a-a36b-d745327d20d9'), SubscriptionInfo(subscription_name='Contoso IT – SH360 - DevTest', subs

In [20]:
# 既存のワークスペースをつなげる
from azureml.core.workspace import Workspace
ws = Workspace.get(name='WS-EastUS',
                      subscription_id='<サブスクリプションID>',
                      resource_group='ML-EastUS'
                     )

In [32]:
# ワークスペース情報をconfigファイルに設定する
ws.write_config()

Wrote the config file config.json to: /home/nbuser/library/training/02.distributed-pytorch-with-horovod/aml_config/config.json


In [33]:
# configファイルから既存のワークスペース情報を取得し、つなげる
ws = Workspace.from_config()

Found the config file in: /home/nbuser/library/training/02.distributed-pytorch-with-horovod/aml_config/config.json


## AML Computeクラスターの作成
Azure Machine Learning Serviceの中で、AML Computeクラスターを作成します。
下記では、既存のクラスターが存在する場合はクラスターを接続、ない場合は新規にクラスターを作成します。

In [21]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           vm_priority="lowpriority",
                                                           max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

Found existing compute target.
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2018-12-23T15:17:38.951000+00:00', 'creationTime': '2018-12-23T15:16:43.882504+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2018-12-23T15:17:47.901864+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6'}


## サンプルコードの用意
ローカル（この例では、Azure Notebookの動いている環境（コンテナー）上）にディレクトリと実際に使うコードを用意します。
この例では、カレントディレクトリの配下にpytorch-distr-hvdを作成します。

In [22]:
import os

project_folder = './pytorch-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

### Pytorch・HorovodのMNISTのサンプルコードをダウンロードします

In [23]:
import shutil

!wget "https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/distributed-pytorch-with-horovod/pytorch_horovod_mnist.py"
shutil.copy('pytorch_horovod_mnist.py', project_folder)

/bin/sh: -c: line 0: syntax error near unexpected token `"https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/distributed-pytorch-with-horovod/pytorch_horovod_mnist.py"'
/bin/sh: -c: line 0: `wget("https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/distributed-pytorch-with-horovod/pytorch_horovod_mnist.py")'


'./pytorch-distr-hvd/pytorch_horovod_mnist.py'

## Experiment (ワークスペース配下のジョブの管理単位）を作成します

In [26]:
from azureml.core import Experiment

experiment_name = 'pytorch-distr-hvd'
experiment = Experiment(ws, name=experiment_name)

## Estimator（ジョブの実行情報やコンテナーの環境情報をまとめて記載した設定）を作成します
現在、ML Serviceでは、TensorFlowとPytorchには専用のEstimatorが用意されています。そのほかのフレームワークに関してはEstimatorからカスタムで作成します。

In [27]:
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=project_folder,
                    compute_target=compute_target,
                    entry_script='pytorch_horovod_mnist.py',
                    node_count=2,
                    process_count_per_node=1,
                    distributed_backend='mpi',
                    use_gpu=True)

## ジョブの実行
上記Estimatorに沿って、下記コマンドでジョブを実行できます。

In [34]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: pytorch-distr-hvd,
Id: pytorch-distr-hvd_1545582966245,
Type: azureml.scriptrun,
Status: Queued)


### ジョブの監視

### 非同期の監視 
10-15秒単位で、非同期にジョブ情報を収集します。グラフィカルな表示とログの表示ができます。

In [36]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

MSI: Failed to retrieve a token from 'http://localhost:25198/nb/api/nbsvc/oauth2/token' with an error of 'HTTPConnectionPool(host='localhost', port=25198): Max retries exceeded with url: /nb/api/nbsvc/oauth2/token (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7bd4e99e48>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))'. This could be caused by the MSI extension not yet fullly provisioned.


### ログの監視
Jupyterをブロックしてログを表示します。

In [ ]:
run.wait_for_completion(show_output=True)